In [1]:
# Import Libraries
import time
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
pd.set_option('display.precision', 0)

In [3]:
# Disable webpage and Image Loading
'''
# Firefox
options = webdriver.FirefoxOptions()
options.add_argument("--headless")  # Enable headless mode
options.set_preference("permissions.default.image", 2)  # Disable images loading
driver = webdriver.Firefox(options=options)
driver.implicitly_wait(5)
'''
# chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)


In [4]:
# Navigate to Google Search
search_keyword = "cerveza"
driver.get("https://www.unimarc.cl/search?q=" + search_keyword)

In [5]:
def scrape_page():

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    
    search_results = soup.find_all('section', class_='baseContainer_container__TSgMX')
    
    data_list = []
    
    for result in search_results:
    
        product_data = {}
    
        try:
            pronto=result.find('p', class_='Text_text--grayNeutral__h7KRC')
            #Text_text--grayNeutral__h7KRC
            #Text_text--none__zez2n
        except:
            continue
    
        try:
            product_description = result.find('p', class_ = 'Shelf_nameProduct__CXI5M')
            product_data['Descripción'] = product_description.get_text(strip=True) 
        except:
            continue
        
        try:
            promotion = result.find('label', class_="ListPrice_priceProduct__6roCh")
            product_data['promocion'] = promotion.get_text(strip=True) if promotion else None
        except:
            continue
        
        sale_price = result.find('p', class_='Text_text--primary__OoK0C')
        product_data['Precio Venta'] = sale_price.get_text(strip=True)  if sale_price else None
        
        reference_price = result.find('p', class_='Text_text--lg__GZWsa')
        product_data['Precio Normal'] = reference_price.get_text(strip=True) if reference_price else None
    
        discount_percentage = result.find('p', class_='OfferLabel_offerLabel__WrBKB')
        product_data['Descuento%'] = discount_percentage.get_text(strip=True) if discount_percentage else None
    
        standard_price = result.find('p', class_='Text_text--none__zez2n', string=lambda text: text and ' x ' in text)
        product_data['Precio Standard'] = standard_price.get_text(strip=True).replace('(', '').replace(')', '') if standard_price else None
        
        data_list.append(product_data)
        
    return data_list


In [6]:
data_list_pages = []
pages_temp=[]

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)
len_pages=int(driver.find_elements(By.CSS_SELECTOR, "div.Pagination_item--base__fM7nj > p")[-1].text)

# Scrape the first page       
data_list_pages += scrape_page()

for i in range(1, len_pages):
       
    # click on the next page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.find_elements(By.CSS_SELECTOR, "div.Pagination_item--base__fM7nj > p")[i].click()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    # Scrape the current page       
    data_list_pages += scrape_page()
    
# close driver
driver.quit()

In [7]:
# Create a DataFrame of Pandas
df = pd.DataFrame(data_list_pages)
# Print the DataFrame
df

,Descripción,promocion,Precio Venta,Precio Normal,Descuento%,Precio Standard
0,Cerveza cristal botella 24 un de 355 cc,2 x,$8.490,$8.490,41%,$996 x L
1,Cerveza cristal botella 24 un de 355 cc,None,$8.490,$8.490,41%,$996 x L
2,Cerveza budweiser zero botella 6 un de 330 cc,2 x,$8.390,$8.390,None,$2.119 x L
3,Cerveza patagonia austral red lager lata 470 cc,4 x,$3.992,$3.992,None,$2.123 x L
4,Cerveza patagonia austral hoppy lager lata 470 cc,4 x,$3.992,$3.992,None,$2.123 x L
...,...,...,...,...,...,...
163,Pack cerveza cusqueña lata 6 un de 354 cc,None,None,None,None,None
164,Pack cerveza stella artois lata 12 un de 354 cc,None,None,None,None,None
165,Pack cerveza stella artois sin alcohol 6 un de...,None,None,None,None,None
166,Pack cerveza austral calafate botella 6 un de ...,None,None,None,None,None


In [8]:
i

3